In [1]:
from collections import defaultdict

## IDEA: Compute retrieved doc score with Reciprocal Rank for all previous turn, then make intersection to top results

In [2]:
#1. loading data: load the first k (e.g. k=10) docs for each utterance in a dictionary

map_utt_docs = defaultdict(list) #dictionary: key = utt_id, value = list of top-k results

max_k = 200

#server
path = "../data/BERT-reranking/"
with open(path+"res_2020_manual.txt.trec", 'r') as f:
    next(f)
    for line in f:
        line_fields = line.rstrip().split(" ")
        utt_id = line_fields[0]
        doc_id = line_fields[2]
        rank = int(line_fields[3])
        if rank <= max_k: 
            map_utt_docs[utt_id].append(doc_id)

In [3]:
map_utt_docs["87_6"]

['CAR_324a662bc5882c7008be352a502ab559cd54bada',
 'CAR_68d93a8ab335006cbf29bdc6dd83fbbd2173850f',
 'MARCO_7016258',
 'MARCO_8513150',
 'MARCO_7016259',
 'CAR_c264ac5e552ea8c67c92d609d4e76b4151bac22c',
 'CAR_b4ee3f01e91b0d92734b8c011a119c58a327bfa6',
 'MARCO_1617391',
 'CAR_17818109b7a798085b34d528c7ab7b62cd95f945',
 'MARCO_7016257',
 'MARCO_7271699',
 'MARCO_7415579',
 'CAR_8f15e0c63548e6c132603b7049609522e5290c9b',
 'MARCO_5492971',
 'CAR_828d1e0e496786c2c0e10ba5aae68360191245f2',
 'CAR_54a248d5d218ece609377b916c23d3039aa48228',
 'MARCO_6315739',
 'MARCO_4542054',
 'MARCO_7415583',
 'MARCO_52429',
 'CAR_296971a09e6c676acc92dd4bd7b1aa6f479bcd5f',
 'MARCO_28182',
 'CAR_e6846e2e19b66e2f79f97744460099390b8305a7',
 'CAR_39f52a72077c519231e73e7605d1c46e84460dca',
 'MARCO_4542058',
 'MARCO_3306011',
 'MARCO_6315738',
 'CAR_e329b876005e0edde984c7e8c8303f8573749a56',
 'MARCO_7642123',
 'MARCO_6315737',
 'CAR_aa2e70dc214b5e9818808b1f8dc5d9aaa306545b',
 'CAR_c08823be13fef5b099f63b3bb28261c0d541d

In [4]:
### REMMBER to check the YEAR

qid_rel_dict = defaultdict(int)
path = "../data/CAST_qrels/"
with open(path+"qrels-docs.2020.txt", 'r') as f:
    for line in f:
        line_fields = line.rstrip().split(" ")
        utt_id = line_fields[0]
        doc_id = line_fields[2]
        relevance = int(line_fields[3])
        if relevance > 0:
            qid_rel_dict[utt_id]+=1

In [5]:
qid_rel_dict["87_6"]

0

In [6]:
#functions to check the intersection between top-k results of current turn and top-k results of the previous turns 

def intersection(lst1, lst2):
    return set(lst1).intersection(lst2)


def compute_overapping_documents_set(conv_id, curr_turn, f, qid_rel_dict, top_k=200):
    
    total_overlapping = 0
    
    utt_id = str(conv_id)+"_"+str(curr_turn)
    docs_current_turn =  map_utt_docs[utt_id]

    docs_so_far = defaultdict(int)
    for turn_id in range(1, curr_turn):
        if str(conv_id)+"_"+str(turn_id) in map_utt_docs:
            my_list = map_utt_docs[str(conv_id)+"_"+str(turn_id)]
            for i, el in enumerate(my_list, start=1):                
                docs_so_far[el] += 1/i
        else:
            f.write(str(conv_id)+"_"+str(turn_id) + " not found\n")
            
    # Top docs based on RR ... so far
    sorted_docs_so_far = sorted(docs_so_far.items(), key=lambda item: item[1], reverse=True)
    top_k_docs_so_far = [x for x,y in sorted_docs_so_far[:top_k]]
    # print(sorted_docs_so_far)
    # print(top_k_docs_so_far)
            
    total_overlapping = len(intersection(top_k_docs_so_far, docs_current_turn))
    # print(docs_so_far, docs_current_turn, total_overlapping)
    normalized_overlapping = "not in qrel"
    if utt_id in qid_rel_dict:
        if qid_rel_dict[utt_id]>0:
            normalized_overlapping = len(intersection(top_k_docs_so_far, docs_current_turn))/qid_rel_dict[utt_id]
        else:
            normalized_overlapping = "0 relevants"
        
    f.write(utt_id+"\t"+str(total_overlapping)+"\t"+str(qid_rel_dict[utt_id])+"\t"+str(normalized_overlapping)+"\n")

In [7]:
# main (with sets, no duplicates): to run after loading data

path = "../data/proof_of_concepts_preliminary_results/retrieved/top200/"
with open(path+"RR-overalapping-sets-docs.res_2020_manual.txt", 'w') as f:
    for el in map_utt_docs:
        conv_id = int(el.split("_")[0])
        curr_turn = int(el.split("_")[1])
        compute_overapping_documents_set(conv_id, curr_turn, f, qid_rel_dict, 200)
        # if curr_turn == 3:
        #     break